In [8]:
import time

import numpy as np
import onnx
import torch
import torchvision
from torchvision import transforms
import tqdm

from furiosa.optimizer import optimize_model
from furiosa.quantizer import quantize, Calibrator, CalibrationMethod
import furiosa.runtime.session

In [13]:
%cd /home/elicer/MeerCam/AI/model_export

/home/elicer/MeerCam/AI/model_export


In [14]:
!python onnx_export.py --weights=/home/elicer/MeerCam/AI/model_export/plzplz.pt --onnx_path=/home/elicer/MeerCam/AI/model_export/plzplz.onnx


Starting ONNX export with onnx 1.14.1...
/home/elicer/.local/lib/python3.10/site-packages/ultralytics/nn/modules/head.py:49: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  elif self.dynamic or self.shape != shape:
/home/elicer/.local/lib/python3.10/site-packages/ultralytics/utils/tal.py:254: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  for i, stride in enumerate(strides):


In [16]:
!onnxsim /home/elicer/MeerCam/AI/model_export/plzplz.onnx /home/elicer/MeerCam/AI/model_export/plzplz.onnx 

Your model contains "Tile" ops or/and "ConstantOfShape" ops. Folding these ops 
can make the simplified model much larger. If it is not expected, please specify
"--no-large-tensor" (which will lose some optimization chances)
Simplifying...
Finish! Here is the difference:
┏━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃                 ┃ Original Model ┃ Simplified Model ┃
┡━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ Add             │ 20             │ 8                │
│ Concat          │ 24             │ 19               │
│ Constant        │ 217            │ 144              │
│ ConstantOfShape │ 3              │ 0                │
│ Conv            │ 64             │ 64               │
│ Div             │ 10             │ 1                │
│ Expand          │ 6              │ 0                │
│ Gather          │ 9              │ 0                │
│ MaxPool         │ 3              │ 3                │
│ Mul             │ 76             │ 58               │


In [10]:
pwd

'/home/elicer/MeerCam/AI/model_export/demo_plz'

In [17]:
!python furiosa_quantizer.py --onnx_path=plzplz.onnx --output_path=plzplz_i8.onnx --calib_data=../val2017

libfuriosa_hal.so --- v0.11.0, built @ 43c901f
Calibration: 100%|█████████████████████████| 100/100 [00:27<00:00,  3.69image/s]
Completed quantinization >> plzplz_i8.onnx


In [18]:
!furiosa-compiler plzplz_i8.onnx -o plzplz_i8.enf --target-npu=warboy

[1/6] 🔍   Compiling from onnx to dfg
Done in 0.5164058s
[2/6] 🔍   Compiling from dfg to ldfg
▸▹▹▹▹ 
▹▸▹▹▹ [1/3] Splitting graph(LAS)...
▹▹▸▹▹ [1/3] Splitting graph(LAS)...
▹▹▹▸▹ [1/3] Splitting graph(LAS)...
▹▹▹▹▸ [1/3] Splitting graph(LAS)...
▹▹▹▹▹ [1/3] Splitting graph(LAS)...
▸▹▹▹▹ [1/3] Splitting graph(LAS)...
▹▸▹▹▹ [1/3] Splitting graph(LAS)...
▹▹▸▹▹ [1/3] Splitting graph(LAS)...
▹▹▹▸▹ [1/3] Splitting graph(LAS)...
▹▹▹▹▸ [1/3] Splitting graph(LAS)...
▹▹▹▹▹ [1/3] Splitting graph(LAS)...
▸▹▹▹▹ [1/3] Splitting graph(LAS)...
▹▸▹▹▹ [1/3] Splitting graph(LAS)...
▹▹▸▹▹ [1/3] Splitting graph(LAS)...
▹▹▹▸▹ [1/3] Splitting graph(LAS)...
▹▹▹▹▸ [1/3] Splitting graph(LAS)...
▹▹▹▹▹ [1/3] Splitting graph(LAS)...
▸▹▹▹▹ [1/3] Splitting graph(LAS)...
▹▸▹▹▹ [1/3] Splitting graph(LAS)...
▹▹▸▹▹ [1/3] Splitting graph(LAS)...
▹▹▹▸▹ [1/3] Splitting graph(LAS)...
▹▹▹▹▸ [1/3] Splitting graph(LAS)...
▹▹▹▹▹ [1/3] Splitting graph(LAS)...
▸▹▹▹▹ [1/3] Splitting graph(LAS)...
▹▸▹▹▹ [1/3] Splitting graph(LAS)...


In [19]:
%cd /home/elicer/MeerCam/AI/model_export/demo_plz

/home/elicer/MeerCam/AI/model_export/demo_plz


In [20]:
pwd

'/home/elicer/MeerCam/AI/model_export/demo_plz'

In [19]:
!pip install --upgrade protobuf

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 7.1 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.21.2
    Uninstalling protobuf-4.21.2:
      Successfully uninstalled protobuf-4.21.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboard 2.15.1 requires protobuf<4.24,>=3.19.6, but you have protobuf 4.25.1 which is incompatible.


In [21]:
!python run_demo.py

2023-11-25T15:37:33.141536Z  INFO furiosa_rt_core::driver::event_driven::coord: FuriosaRT (v0.10.3, rev: 394c19392, built at: 2023-11-22T08:53:04Z) bootstrapping ...
2023-11-25T15:37:33.144470Z  INFO furiosa_rt_core::driver::event_driven::coord: Found furiosa-compiler (v0.10.1, rev: 8b00177, built at: 2023-11-23T02:22:00Z)
2023-11-25T15:37:33.144494Z  INFO furiosa_rt_core::driver::event_driven::coord: Found libhal (type: warboy, v0.12.0, rev: 56530c0 built at: 2023-11-16T12:37:25Z)
2023-11-25T15:37:33.144506Z  INFO furiosa_rt_core::driver::event_driven::coord: [Runtime-0] detected 1 NPU device(s):
2023-11-25T15:37:33.169009Z  INFO furiosa_rt_core::driver::event_driven::coord: - [0] npu:0:1 (warboy-b0, 64dpes, firmware: 1.7.8, e9f371e)
2023-11-25T15:37:33.169148Z  INFO furiosa_rt_core::driver::event_driven::coord: [Runtime-0] started
2023-11-25T15:37:33.171144Z  INFO furiosa::runtime: Saving the compilation log into /home/elicer/.local/state/furiosa/logs/compiler-20231125153733-k1cb64.l

In [5]:
!furiosactl ps

+-----+-----+-----+
| NPU | PID | CMD |
+-----+-----+-----+


In [6]:
!furiosactl list

+------+------------------------------+-----------------------------+
| NPU  | Cores                        | DEVFILES                    |
+------+------------------------------+-----------------------------+
| npu0 | 0 (available), 1 (available) | npu0pe0, npu0pe1, npu0pe0-1 |
+------+------------------------------+-----------------------------+
